In [1]:
# Import python modules
import tensorflow 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import keras_tuner 
from google.cloud import aiplatform

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy
import pandas
import json, os

In [2]:
# Declare variables
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
# PROJECT_ID = "hzchen-lowa"
# MODEL_PATH='gs://'+PROJECT_ID+'-bucket/model/'
# DATASET_PATH='gs://'+PROJECT_ID+'/area_cover_dataset.csv'
# PIPELINE_ROOT = 'gs://'+PROJECT_ID
# MODEL_ARTIFACTS_LOCATION ='gs://'+PROJECT_ID+'-bucket/'

MODEL_PATH='gs://'+"hzchen-lowa"+'-bucket/model/'
DATASET_PATH='gs://'+"hzchen-lowa"+'/area_cover_dataset.csv'
PIPELINE_ROOT = 'gs://'+"hzchen-lowa"
MODEL_ARTIFACTS_LOCATION ='gs://'+"hzchen-lowa"+'-bucket/'

# MODEL_PATH='gs://hzchen-lowa/cepf005/staging/model/'
# DATASET_PATH='gs://hzchen-lowa/cepf005/area_cover_dataset.csv'
# PIPELINE_ROOT = 'gs://hzchen-lowa/cepf005/'
# MODEL_ARTIFACTS_LOCATION ='gs://hzchen-lowa/cepf005/staging/'

In [3]:
# Read the area_cover_dataset csv data into pandas dataframe
area_cover_dataframe = pandas.read_csv(DATASET_PATH)

**Task 4** Create the function that converts categorical data to indexed integer values

In [4]:
# Function that takes the area cover dataframe and converts the two categorical (string) columns into indexed values
def index(dataframe):
    
    dataframe['Wilderness_Area'] = dataframe['Wilderness_Area'].astype('category').cat.codes
    dataframe['Soil_Type'] = dataframe['Soil_Type'].astype('category').cat.codes
    
    return dataframe

In [5]:
indexed_dataframe = index(area_cover_dataframe)
features_dataframe = indexed_dataframe.drop("Area_Cover", axis = 1)

**Task 5** Extract the feature columns and standardize the values

In [6]:
# Extract the feature columns into a new dataframe called scaler_features that has been standardized using the sklearn.preprocessing.StandardScaler method.
# The features are all columns from the area cover dataset except the "Area_Cover" column
indexed_dataframe = index(area_cover_dataframe)
features_dataframe = indexed_dataframe.drop("Area_Cover", axis = 1)
standard_scaler = StandardScaler()

scaled_features = standard_scaler.fit_transform(features_dataframe)

In [7]:
# Create a binary matrix containing the categorical Area_Cover column data converted using keras.utils.to_categorical()
labels_dataframe = indexed_dataframe["Area_Cover"]
categorical_labels = to_categorical(labels_dataframe)

In [8]:
# Split the dataset into model training and validation data
# dfx_train, dfx_val, dfy_train, dfy_val = train_test_split(scaled_features.values, categorical_labels, test_size=0.2)

dfx_train, dfx_val, dfy_train, dfy_val = train_test_split(scaled_features, categorical_labels, test_size=0.2)

**Task 6** Create a function that returns a sequential categorical model function with a hyperparameter tuning layer

In [9]:
# Create a function that returns a sequential categorical model function with a hyperparameter tuning layer
def build_model(hptune):
    model = Sequential()
    model.add(Dense(128, input_shape = (12,), activation = "relu"))
    
    model.add(
        Dense(
            # Define the hyperparameter.
            units=hptune.Int("units", min_value=2, max_value=12, step=1),
            activation="relu",
        )
    )
    model.add(Dense(7, activation="softmax"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
    )
    
    return model

**Task 7** Create a Keras Hyperband Hyperparameter tuner with an accuracy objective

In [10]:
# Create a Keras Hyperband Hyperparameter tuner with an accuracy objective

tuner = keras_tuner.Hyperband(build_model,
                            objective='val_accuracy',
                            max_epochs=20,
                            factor=3,
                            directory='hpo_logs',
                            project_name='cepf005')

INFO:tensorflow:Reloading Oracle from existing project hpo_logs/cepf005/oracle.json
INFO:tensorflow:Reloading Tuner from hpo_logs/cepf005/tuner0.json


2022-06-20 00:48:37.894215: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-20 00:48:37.894295: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gcplow01): /proc/driver/nvidia/version does not exist
2022-06-20 00:48:37.898038: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Task 8** Perform Hyperparameter tuning and train the optimal model

You do not have to add any of your own code for this task. Run the cells to tune, optimize and train the model. 

In [11]:
# Define an early stopping callback using that stops when the validation loss quantity does not improve after 5 epochs
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Perform a Keras Tuner Search for the best hyperparameter configurations using the training data split over 50 epochs
tuner.search(dfx_train, dfy_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters for the model as determined from the search
best_hyperparameters=tuner.get_best_hyperparameters(num_trials=10)[0]

INFO:tensorflow:Oracle triggered exit


In [12]:
# Create a new model using the best_hyperparameters and train it. 
model = tuner.hypermodel.build(best_hyperparameters)
history = model.fit(dfx_train, dfy_train, epochs=50, validation_split=0.2)

Epoch 1/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.7388 - accuracy: 0.6973 - val_loss: 0.6447 - val_accuracy: 0.7286
Epoch 2/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6272 - accuracy: 0.7346 - val_loss: 0.6059 - val_accuracy: 0.7408
Epoch 3/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6038 - accuracy: 0.7420 - val_loss: 0.5888 - val_accuracy: 0.7506
Epoch 4/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5872 - accuracy: 0.7492 - val_loss: 0.5775 - val_accuracy: 0.7530
Epoch 5/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5731 - accuracy: 0.7537 - val_loss: 0.5597 - val_accuracy: 0.7600
Epoch 6/50
2000/2000 [==============================] - 9s 4ms/step - loss: 0.5610 - accuracy: 0.7601 - val_loss: 0.5532 - val_accuracy: 0.7574
Epoch 7/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.5511 - accuracy: 0.7628 - val_loss: 0.5468 - val_accuracy:

In [13]:
# Using the model training history find and print out the epoch with the best validation accuracy. 
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 46


In [14]:
# Print out the Model test loss and test accuracy by evaluating the validation data split. 
eval_result = model.evaluate(dfx_val, dfy_val)
print("[Model test loss, test accuracy]:", eval_result)

625/625 [==============================] - 1s 1ms/step - loss: 0.4514 - accuracy: 0.8124
[Model test loss, test accuracy]: [0.45144736766815186, 0.8123999834060669]


In [15]:
# Create a new model (hypermodel) using the best_hyperparameters and retrain. 
hypermodel = tuner.hypermodel.build(best_hyperparameters)
# Retrain the model using the number of epochs that was previously determined to be the best. 
hypermodel.fit(dfx_train, dfy_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/46
2000/2000 [==============================] - 5s 2ms/step - loss: 0.7302 - accuracy: 0.6989 - val_loss: 0.6396 - val_accuracy: 0.7293
Epoch 2/46
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6204 - accuracy: 0.7365 - val_loss: 0.6061 - val_accuracy: 0.7431
Epoch 3/46
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5999 - accuracy: 0.7445 - val_loss: 0.5873 - val_accuracy: 0.7474
Epoch 4/46
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5846 - accuracy: 0.7497 - val_loss: 0.5752 - val_accuracy: 0.7561
Epoch 5/46
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5714 - accuracy: 0.7555 - val_loss: 0.5681 - val_accuracy: 0.7564
Epoch 6/46
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5591 - accuracy: 0.7606 - val_loss: 0.5517 - val_accuracy: 0.7663
Epoch 7/46
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5495 - accuracy: 0.7634 - val_loss: 0.5402 - val_accuracy:

In [16]:
# Print out the test loss and test accuracy for hypermodel by evaluating the validation data split. 
eval_result = hypermodel.evaluate(dfx_val, dfy_val)
print("[Hypermodel test loss, test accuracy]:", eval_result)

625/625 [==============================] - 1s 1ms/step - loss: 0.4747 - accuracy: 0.8004
[Hypermodel test loss, test accuracy]: [0.47471776604652405, 0.8004000186920166]


In [17]:
# Save the hypertuned model
# NB the MODEL_PATH bucket must be created before this will succeed and it must be in the same location as the model.
# e.g. gsutil mb -l us-central1  gs://${PROJECT_ID}-bucket
hypermodel.save(MODEL_PATH)

2022-06-20 00:55:38.837569: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://hzchen-lowa-bucket/model/assets


**Task 9** Create a Custom Container for Vertex AI pipeline model training
1. Create a Python model trainer module using the above code
2. Save the code as `model.py` in the `model/trainer` beneath the current working directory for this notebook
3. Make sure you set the Project ID correctly in the Python script. 
4. Create the Dockerfile definition in the `model/` directory for your custom training container using the `gcr.io/deeplearning-platform-release/tf2-cpu.2-6` base container image

Once you have prepared the custom container Python module code and Dockerfile you can build and test the custom container. 

In [ ]:
# Build the container using the following gcr.io tag
IMAGE_URI="gcr.io/{}/tensorflow:latest".format(PROJECT_ID)
!docker build ./model/. -t $IMAGE_URI

In [ ]:
# Run the docker image locally to test it
!docker run $IMAGE_URI

In [ ]:
# Push the docker image to the Google container registry
!docker push $IMAGE_URI

In [ ]:
# Install kubeflow pipeline SDK and google cloud pipeline component for building Vertex AI pipelines
!pip3 install kfp google_cloud_pipeline_components

In [18]:
# Import the libraries required for Vertext AI pipelines
import kfp
from kfp.v2 import compiler
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [ ]:
# bucket = MODEL_ARTIFACTS_LOCATION
# project = PROJECT_ID
# gcp_region = REGION
# container_uri = "gcr.io/dave-selfstudy01/tensorflow:latest"

# training_op = gcc_aip.CustomContainerTrainingJobRunOp(
#     display_name="tensorflow-train-model",
#     container_uri=container_uri,
#     project=project,
#     location=gcp_region,
#     staging_bucket=bucket,
#     training_fraction_split=0.8,
#     validation_fraction_split=0.1,
#     test_fraction_split=0.1,
#     model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest",
#     machine_type="n1-standard-4"       
# )

# training_op

In [ ]:
# training_op.outputs["model"]

**Task 10** Define the Vertex AI Training pipeline

1. Add your code for the Training Operation using your newly created custom container
    * This should reference the custom container_uri passed in as a parameter
    * This should use "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest" for the `model_serving_container_image_uri`
2. Add your code for the Model Deploy Operation
    * This operation should output a model and an endpoint.
    
All machine types should be specified as "n1-standard-4"


In [83]:
# Define the Vertex AI pipeline
@kfp.dsl.pipeline(name="vertex-ai-pipeline",
                  pipeline_root=PIPELINE_ROOT)
def pipeline(
    bucket: str = MODEL_ARTIFACTS_LOCATION,
    project: str = PROJECT_ID,
    gcp_region: str = REGION,
    container_uri: str = "",
):
    
    training_op = gcc_aip.CustomContainerTrainingJobRunOp(
        display_name="tensorflow-train-model",
        container_uri=container_uri,
        project=project,
        location=gcp_region,
        staging_bucket=bucket,
        training_fraction_split=0.8,
        validation_fraction_split=0.1,
        test_fraction_split=0.1,
        model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest",
        machine_type="n1-standard-4",
        base_output_dir=bucket,
    )
       
    create_endpoint_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name = "tensorflow-model-endpoint",
    )
    
    model_deploy_op = gcc_aip.ModelDeployOp(        
        endpoint=create_endpoint_op.outputs["endpoint"],
        model=training_op.outputs["model"],
        dedicated_resources_min_replica_count=2,
        dedicated_resources_max_replica_count=10,
        dedicated_resources_machine_type="n1-standard-4",
        traffic_split={"0": 100}
    )

In [84]:
# Compile the  Vertex AI pipeline
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="pipeline.json"
)

/opt/conda/envs/gcp/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


**Task 11** Create the Vertex AI Pipeline job object

The pipeline job must specified using the compiled pipeline definition JSON file and should point to your saved model location and your custom training container

In [85]:
# Create the Vertex AI Pipeline job object
pipeline_job = aiplatform.PipelineJob(  
    display_name = "cepf005_pipeline",
    template_path = "pipeline.json",
    parameter_values = {"container_uri": "gcr.io/dave-selfstudy01/tensorflow:latest"}
)

In [86]:
# import google.auth

# credentials, project_id = google.auth.default()
# credentials, project_id

In [87]:
# credentials.service_account_email

In [88]:
# Run the Vertex AI pipeline job
pipeline_job.run(service_account="dave-selfstudy-demo-783@dave-selfstudy01.iam.gserviceaccount.com")
# pipeline_job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/886724937990/locations/us-central1/pipelineJobs/vertex-ai-pipeline-20220620021533
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/886724937990/locations/us-central1/pipelineJobs/vertex-ai-pipeline-20220620021533')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/vertex-ai-pipeline-20220620021533?project=886724937990
PipelineJob projects/886724937990/locations/us-central1/pipelineJobs/vertex-ai-pipeline-20220620021533 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/886724937990/locations/us-central1/pipelineJobs/vertex-ai-pipeline-20220620021533 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/886724937990/locations/us-central1/pipelineJobs/vertex-ai-pipeline-20220620021533 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/886724937990/locations/us-centra

In [89]:
# List the model created by the pipeline
!gcloud ai models list --region=$REGION

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
MODEL_ID             DISPLAY_NAME
4895962550765617152  tensorflow-train-model-model
2000504232133787648  LEYI_VIP_20219175857
3060538994426118144  SKAB-BQML-Kmeans01
87037340454748160    SKAB-AutoML-import3
8180005870839529472  SKAB-AutoML-import2-bad
4879993243883798528  SKAB-AutoML-import-bad
2826351813802852352  SKAB-clean-GS_202182954748


In [90]:
# Store the endpoint ID where the model has been deployed 

!gcloud ai endpoints list --region=$REGION
ENDPOINT_IDS=!gcloud ai endpoints list --region=$REGION --format="value(name)" 2>/dev/null
print("Vertex AI Endpoint ID:" + ENDPOINT_IDS[0])


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
5757737774383890432  tensorflow-model-endpoint
349433591400235008   SKAB-BQML-Kmeans-endpoint01
5352932777408856064  SKAB-AutoML-endpoint
Vertex AI Endpoint ID:5757737774383890432


In [45]:
# Copy in the pre-prepared sample test.json 
!gsutil cp gs://sureskills-lab-dev/CEPF/vertex-ai/test.json . 

Copying gs://sureskills-lab-dev/CEPF/vertex-ai/test.json...
/ [1 files][ 22.0 KiB/ 22.0 KiB]                                                
Operation completed over 1 objects/22.0 KiB.                                     


**Task 12** Create a function to convert the source JSON test data to an array of normalized column values

The test data consists of samples with feature data that you want to use to generate area_cover type predictions using the model endpoint. 

You must define a functon that performs the following tasks:
1. Read the `test.json` instance data into a dataframe
2. Normalize the column data using the `StandardScalar.fit_transform` method
3. Output an array of arrays containing the normalized feature column data for each test instance.

In [128]:
# Convert the json test data to an array of standard scaler normalized column data

def get_instances(file_name):
    # instances = []

    df = pandas.read_json('test.json')
    df['Wilderness_Area'] = df.Wilderness_Area.astype('category').cat.codes
    df['Soil_Type'] = df.Soil_Type.astype('category').cat.codes
    standard_scaler = StandardScaler()
    instances = standard_scaler.fit_transform(df).tolist()

    # normalize_df = df

    # for _ in normalize_df.values:
    #     instances.append(list(_))
    return instances

In [119]:
# FILE_NAME = "test.json"
# instances = get_instances(FILE_NAME)
# instances

In [120]:
# preds = hypermodel.predict(instances)
# import numpy as np
# np.argmax(preds, axis=1) 

In [121]:
# from google.protobuf import json_format
# from google.protobuf.struct_pb2 import Value

In [122]:
# test_data = [json_format.ParseDict(instance_dict, Value()) for instance_dict in instances.tolist()]

In [123]:
# test_data

In [124]:
# Define a function for making predictions using the endpoint
def endpoint_predict(project: str, location: str, instances, endpoint: str):
    aiplatform.init(project=project, location=location)
    endpoint = aiplatform.Endpoint(endpoint)   
    prediction = endpoint.predict(instances=instances)
    return prediction

In [129]:
# Test the result by calling get_values() that convert JSON to the numpy array
# Replace the endpoint ID with the new ENDPOINT_ID if needed
FILE_NAME = "test.json"
instances = get_instances(FILE_NAME)
prediction_result = endpoint_predict(
    project=PROJECT_ID,
    location=REGION,
    instances=instances,
    endpoint=ENDPOINT_IDS[0]
)

In [130]:
# prediction_result

Prediction(predictions=[[0.93716383, 0.0621097945, 1.37595324e-09, 1.99738046e-21, 0.000645427383, 2.18312968e-09, 8.10352576e-05], [0.0578953698, 0.942091, 1.51431504e-12, 6.53345959e-34, 1.2443269e-05, 1.24841421e-13, 1.18084336e-06], [0.759457111, 0.239930719, 2.94277047e-09, 7.90986776e-21, 0.000612155942, 2.54897294e-08, 7.65784325e-09], [0.457372069, 0.541426063, 1.50434101e-08, 3.81425304e-22, 0.000884396723, 2.42730702e-09, 0.000317476864], [0.37649858, 0.6003474, 8.25667448e-05, 3.89687761e-11, 0.0228658486, 0.000204873984, 6.49348749e-07], [0.0305828564, 0.967416763, 1.53155817e-08, 1.53276195e-23, 0.00189073232, 4.66670436e-09, 0.000109669934], [0.0786813796, 0.921168208, 5.83852289e-14, 1.86628517e-37, 3.45642366e-05, 1.26103733e-14, 0.000115819705], [0.178011775, 0.817611516, 8.14428436e-09, 7.71439859e-24, 0.00160957943, 1.20851651e-09, 0.00276722759], [0.269340038, 0.0002835372, 3.97127307e-32, 0.0, 3.54522381e-21, 1.08376845e-27, 0.730376363], [0.0746046901, 0.924553931

In [131]:
# Save `Area_Cover` predictions with respect to the test instance features
area_cover_predictions={}
for index,area_cover in enumerate(prediction_result.predictions):
    print(index,":",numpy.argmax(area_cover), end=' \n')
    area_cover_predictions[index]=str(numpy.argmax(area_cover))
    
f = open("predictions.txt", "w")
f.write(json.dumps(area_cover_predictions))
f.close()

0 : 0 
1 : 1 
2 : 0 
3 : 1 
4 : 1 
5 : 1 
6 : 1 
7 : 1 
8 : 6 
9 : 1 
10 : 1 
11 : 0 
12 : 0 
13 : 0 
14 : 5 
15 : 0 
16 : 1 
17 : 1 
18 : 0 
19 : 6 
20 : 1 
21 : 6 
22 : 1 
23 : 0 
24 : 1 
25 : 0 
26 : 5 
27 : 0 
28 : 0 
29 : 0 
30 : 0 
31 : 2 
32 : 2 
33 : 1 
34 : 1 
35 : 1 
36 : 0 
37 : 2 
38 : 1 
39 : 1 
40 : 1 
41 : 1 
42 : 1 
43 : 0 
44 : 0 
45 : 1 
46 : 1 
47 : 1 
48 : 0 
49 : 0 
